In [92]:
import pyomo.environ as pyo
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
import pandas as pd

Import electricity demand, capacity factors, time step duration and power plant data 

In [93]:
demand = pd.read_csv('load.csv',header=None)
demand.rename(columns={0:"timestep", 1:"load_MW"}, inplace=True)
demand

,timestep,load_MW
0,t1,83115
1,t2,71169
2,t3,66729
3,t4,61442
4,t5,60430
5,t6,57013
6,t7,52048
7,t8,48701
8,t9,43981
9,t10,40498


In [94]:
cf = pd.read_csv('capacity_factors.csv')
cf.rename(columns={cf.columns[0]:"tech"}, inplace=True)
cf[["t1", "t2", "t3", "t4", "t5","t6", "t7", "t8", "t9", "t10"]] = cf[["t1", "t2", "t3", "t4", "t5","t6", "t7", "t8", "t9", "t10"]].apply(pd.to_numeric)
cf

,tech,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
0,CCGT,0.59,0.63,0.59,0.61,0.67,0.61,0.63,0.67,0.62,0.64
1,GT_GasOil,0.63,0.69,0.68,0.68,0.70,0.63,0.68,0.61,0.70,0.70
2,Hydro,0.78,0.79,0.69,0.74,0.71,0.83,0.80,0.70,0.78,0.82
3,Coal,0.82,0.92,0.88,0.82,0.93,0.83,0.82,0.82,0.80,0.85
4,Lignite,0.81,0.86,0.77,0.77,0.82,0.81,0.84,0.77,0.86,0.86
5,Nuclear,0.95,0.96,0.87,0.89,0.92,0.94,0.97,0.87,0.88,0.92
6,Wind,0.17,0.16,0.20,0.21,0.31,0.31,0.21,0.24,0.34,0.17
7,Solar,0.24,0.05,0.09,0.17,0.16,0.15,0.12,0.03,0.00,0.00


In [95]:
duration = pd.read_csv('duration.csv',header=None)
duration.rename(columns={0:"timestep", 1:"length"}, inplace=True)
duration

,timestep,length
0,t1,102
1,t2,962
2,t3,962
3,t4,962
4,t5,962
5,t6,962
6,t7,962
7,t8,962
8,t9,962
9,t10,962


In [96]:
tech_data = pd.read_csv('tech_data.csv',header=None)
tech_data.rename(columns={0:"tech", 1:"cap_MW",2:"eta",3:"fuel_p",4:"c_var_other",5:"emf"}, inplace=True)
tech_data.drop([0,1],inplace=True)
tech_data[["cap_MW", "eta", "fuel_p", "c_var_other", "emf"]] = tech_data[["cap_MW", "eta", "fuel_p", "c_var_other", "emf"]].apply(pd.to_numeric)
tech_data.reset_index(drop=True, inplace=True)
tech_data

,tech,cap_MW,eta,fuel_p,c_var_other,emf
0,CCGT,30000,0.54,19.0,1.5,0.2048
1,GT_GasOil,4400,0.28,19.0,1.5,0.2048
2,Hydro,5200,1.00,0.0,1.5,0.0000
3,Coal,22500,0.42,7.4,2.6,0.3420
4,Lignite,21000,0.37,3.4,3.0,0.3996
5,Nuclear,8400,0.33,1.8,0.7,0.0000
6,Wind,61000,1.00,0.0,1.4,0.0000
7,Solar,46500,1.00,0.0,1.0,0.0000


Create a generic model `m` for all timesteps

In [97]:
# Create a concrete Pyomo model
m = pyo.ConcreteModel()
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT) #specify dual variables (shadow prices)

Create a set $S$ for the technologies:

In [98]:
m.S = pyo.Set(initialize=tech_data['tech'])

In [99]:
m.S.pprint()

S : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    8 : {'CCGT', 'GT_GasOil', 'Hydro', 'Coal', 'Lignite', 'Nuclear', 'Wind', 'Solar'}


Define the decision variable generators

In [100]:
# Define the decision variable for each generator
m.generators = Var(m.S, domain=pyo.NonNegativeReals)

In [101]:
m.generators.pprint()

generators : Size=8, Index=S
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
         CCGT :     0 :  None :  None : False :  True : NonNegativeReals
         Coal :     0 :  None :  None : False :  True : NonNegativeReals
    GT_GasOil :     0 :  None :  None : False :  True : NonNegativeReals
        Hydro :     0 :  None :  None : False :  True : NonNegativeReals
      Lignite :     0 :  None :  None : False :  True : NonNegativeReals
      Nuclear :     0 :  None :  None : False :  True : NonNegativeReals
        Solar :     0 :  None :  None : False :  True : NonNegativeReals
         Wind :     0 :  None :  None : False :  True : NonNegativeReals


Define the marginal cost for each generator

In [102]:
# Define the cost functions for each generator
def marginal_price(generator,co2_price):
    """
    Returns the marginal price for the given generator in €/MWh, based on the fuel price, effficiency, further variable costs, and carbon emissions of that generator

    Parameters:
        generator: generator in question
        co2_price: CO2 price in €/tCO2

    Returns:
        marginal_price: marginal price in €/MWh      
    """
    df = tech_data[tech_data['tech'] == generator]
    return round((df['fuel_p']/df['eta'] + df['c_var_other'] + co2_price*df['emf']/df['eta']).values[0],2)


### 1. No CO2 price

Calculate the marginal cost with no CO2 price

In [103]:
tech_data_noCO2price = tech_data.copy()
tech_data_noCO2price['marginal_cost'] = round(tech_data_noCO2price['fuel_p']/tech_data_noCO2price['eta']+tech_data_noCO2price['c_var_other'],2)
tech_data_noCO2price

,tech,cap_MW,eta,fuel_p,c_var_other,emf,marginal_cost
0,CCGT,30000,0.54,19.0,1.5,0.2048,36.69
1,GT_GasOil,4400,0.28,19.0,1.5,0.2048,69.36
2,Hydro,5200,1.00,0.0,1.5,0.0000,1.50
3,Coal,22500,0.42,7.4,2.6,0.3420,20.22
4,Lignite,21000,0.37,3.4,3.0,0.3996,12.19
5,Nuclear,8400,0.33,1.8,0.7,0.0000,6.15
6,Wind,61000,1.00,0.0,1.4,0.0000,1.40
7,Solar,46500,1.00,0.0,1.0,0.0000,1.00


Clone `m` for no CO2 price models `m_0`

In [104]:
m_0 = m.clone()

Define the objective function: we want to minimise the operating costs of the electricity system while meeting the demand for each timestep

\begin{equation}
    \min_{g_{s,t}} \sum_s o_s g_{s,t} \quad \forall t
  \end{equation}

  such that
  
  \begin{align}
    g_{s,t} &\leq G_{s,t} \\
    g_{s,t} &\geq 0 \\
    \sum_s g_{s,t} &= d_t
  \end{align}

In [105]:
# Define the objective function
m_0.cost = Objective(expr=sum(marginal_price(s,0)*m_0.generators[s] for s in m_0.S),
                      sense=pyo.minimize)

In [106]:
m_0.cost.pprint()

cost : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 36.69*generators[CCGT] + 69.36*generators[GT_GasOil] + 1.5*generators[Hydro] + 20.22*generators[Coal] + 12.19*generators[Lignite] + 6.15*generators[Nuclear] + 1.4*generators[Wind] + generators[Solar]


Create dictionary of models for all timesteps

In [107]:
models = {}
for i in range(len(duration)):
    models[duration['timestep'][i]] = m_0.clone() #clone original model for each of the timesteps

#create a list of timesteps
timesteps = list(models.keys())

Each timestep now has a model

In [108]:
models

{'t1': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354225180>,
 't2': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354137a90>,
 't3': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3541eb6d0>,
 't4': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3541f2130>,
 't5': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3542374f0>,
 't6': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354243220>,
 't7': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3542435e0>,
 't8': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354243900>,
 't9': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354243cc0>,
 't10': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3542480e0>}

Add demand and generator limit constraints to all timesteps

  \begin{align}
    g_{s,t} &\leq G_{s,t} \\
    g_{s,t} &\geq 0 \\
    \sum_s g_{s,t} &= d_t
  \end{align}

In [109]:
for i in timesteps:
    model = models[i]
    #add generator limit: capacity*cf at a given timestep
    
    @model.Constraint(model.S)
    def generator_limit(model, s):
        return model.generators[s] <= cf[cf['tech'] == s][i].values[0]*tech_data[tech_data['tech'] == s].cap_MW.values[0] 
    models[i] = model
    
    #add demand constraint: sum of generation must equal demand    
    models[i].demand_constraint = Constraint(expr=sum(models[i].generators[s] for s in models[i].S) == demand[demand.timestep == i].load_MW.values[0]) 

Solve the models at all timesteps

In [110]:
for i in timesteps:
    SolverFactory('cbc').solve(models[i]).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1178175.31
  Upper bound: 1178175.31
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 9
  Number of nonzeros: 8
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.45
  Wallclock time: 0.45
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number o

Extract optimum dispatch in MW for each timestep from the results

In [111]:
dispatch = pd.Series(models[timesteps[0]].generators.get_values(),name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    d = pd.Series(models[i].generators.get_values(),name=i).to_frame()
    dispatch = dispatch.join(d)
dispatch #MW

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,14089.0,8152.0,3478.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GT_GasOil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hydro,4056.0,4108.0,3588.0,3848.0,3692.0,4316.0,4160.0,3640.0,4056.0,4264.0
Coal,18450.0,20700.0,19800.0,13233.0,5440.0,1906.0,3710.0,5548.0,0.0,76.0
Lignite,17010.0,18060.0,16170.0,16170.0,17220.0,17010.0,17640.0,16170.0,11793.0,18060.0
Nuclear,7980.0,8064.0,7308.0,7476.0,7728.0,7896.0,8148.0,7308.0,7392.0,7728.0
Wind,10370.0,9760.0,12200.0,12810.0,18910.0,18910.0,12810.0,14640.0,20740.0,10370.0
Solar,11160.0,2325.0,4185.0,7905.0,7440.0,6975.0,5580.0,1395.0,0.0,0.0


In [112]:
import os  
os.makedirs('results', exist_ok=True)  
dispatch.to_csv('results/dispatch.csv')

Calculate dispatch expenditure [€] for each timestep: electricity price [€/MWh] * load [MW] * length of timestep [h]

In [113]:
dispatch_costs = duration.copy()
dispatch_costs['load_MW'] = demand['load_MW']
dispatch_costs['electricity_price'] = [models[t].dual[models[t].demand_constraint] for t in timesteps]
dispatch_costs['expenditure'] = [models[t].cost.expr() for t in timesteps]*dispatch_costs['length']
dispatch_costs.to_csv('results/dispatch_costs.csv')
dispatch_costs


,timestep,length,load_MW,electricity_price,expenditure
0,t1,102,83115,36.69,1.201739e+08
1,t2,962,71169,36.69,9.711841e+08
2,t3,962,66729,36.69,7.663940e+08
3,t4,962,61442,20.22,5.216656e+08
4,t5,962,60430,20.22,3.914261e+08
5,t6,962,57013,20.22,3.216684e+08
6,t7,962,52048,20.22,3.558553e+08
7,t8,962,48701,20.22,3.670877e+08
8,t9,962,43981,12.19,2.158126e+08
9,t10,962,40498,20.22,2.791044e+08


Calculate total annual expenditure and average electricity price

In [114]:
print("Total annual expenditure [billion €]:", round((dispatch_costs.expenditure).sum()/1e9,3))
print("Average electricity price [€/MWh]:", round(((dispatch_costs.electricity_price*dispatch_costs.length).sum()/dispatch_costs.length.sum()),2))

Total annual expenditure [billion €]: 4.31
Average electricity price [€/MWh]: 23.15


Calculate dispatch emissions per timestep in tCO2: (EMF [tCO2/MWh_th]/eta [MW_el/MW_th] * dispatch [MW])) * length of timestep [h]

In [115]:
dispatch_emissions = dispatch.copy()

In [116]:
for i in timesteps:
    dispatch_emissions[i] = (tech_data.set_index(dispatch.index)['emf'].values*dispatch[i]/tech_data.set_index(dispatch.index)['eta'])*duration[duration['timestep'] == 't1'].length.values[0]


dispatch_emissions.to_csv('results/dispatch_emissions.csv')
dispatch_emissions  #total emissions per timestep, tCO2

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,5.450251e+05,3.153556e+05,1.345445e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.00,0.000000e+00
GT_GasOil,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.00,0.000000e+00
Hydro,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.00,0.000000e+00
Coal,1.532404e+06,1.719283e+06,1.644531e+06,1.099095e+06,4.518309e+05,1.583069e+05,308142.0,4.608010e+05,0.00,6.312343e+03
Lignite,1.873822e+06,1.989490e+06,1.781287e+06,1.781287e+06,1.896955e+06,1.873822e+06,1943222.4,1.781287e+06,1299116.88,1.989490e+06
Nuclear,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.00,0.000000e+00
Wind,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.00,0.000000e+00
Solar,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.00,0.000000e+00


In [117]:
print('Total emissions [million tCO2]:', round(dispatch_emissions.sum().sum()/1e6,2))

Total emissions [million tCO2]: 26.59


Calculate dual prices/shadow prices for each generator for all the timesteps: (how much more expensive the objective function would be if you increased the capacity by one unit)

In [118]:
dual_prices = pd.Series({s: models[timesteps[0]].dual[models[timesteps[0]].generator_limit[s]] for s in models[timesteps[0]].S},name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    dp = pd.Series({s: models[i].dual[models[i].generator_limit[s]] for s in models[i].S},name=i).to_frame()
    dual_prices = dual_prices.join(dp)

dual_prices.to_csv('results/dual_prices.csv')
dual_prices

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GT_GasOil,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Hydro,-35.19,-35.19,-35.19,-18.72,-18.72,-18.72,-18.72,-18.72,-10.69,-18.72
Coal,-16.47,-16.47,-16.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Lignite,-24.50,-24.50,-24.50,-8.03,-8.03,-8.03,-8.03,-8.03,0.00,-8.03
Nuclear,-30.54,-30.54,-30.54,-14.07,-14.07,-14.07,-14.07,-14.07,-6.04,-14.07
Wind,-35.29,-35.29,-35.29,-18.82,-18.82,-18.82,-18.82,-18.82,-10.79,-18.82
Solar,-35.69,-35.69,-35.69,-19.22,-19.22,-19.22,-19.22,-19.22,-11.19,-19.22


### CO2 price of 120 Euros/tCO2

Calculate the marginal cost with CO2 price

In [119]:
tech_data_CO2price = tech_data.copy()
tech_data_CO2price['marginal_cost'] = round(tech_data_noCO2price['fuel_p']/tech_data_noCO2price['eta']+tech_data_noCO2price['c_var_other'] + 120*tech_data_noCO2price['emf']/tech_data_noCO2price['eta'],2)
tech_data_CO2price

,tech,cap_MW,eta,fuel_p,c_var_other,emf,marginal_cost
0,CCGT,30000,0.54,19.0,1.5,0.2048,82.20
1,GT_GasOil,4400,0.28,19.0,1.5,0.2048,157.13
2,Hydro,5200,1.00,0.0,1.5,0.0000,1.50
3,Coal,22500,0.42,7.4,2.6,0.3420,117.93
4,Lignite,21000,0.37,3.4,3.0,0.3996,141.79
5,Nuclear,8400,0.33,1.8,0.7,0.0000,6.15
6,Wind,61000,1.00,0.0,1.4,0.0000,1.40
7,Solar,46500,1.00,0.0,1.0,0.0000,1.00


Clone generic `m` to create model `m_co2`

In [120]:
m_co2 = m.clone()

Define the objective function: we want to minimise the operating costs of the electricity system while meeting the demand for each timestep

\begin{equation}
    \min_{g_{s,t}} \sum_s o_s g_{s,t} \quad \forall t
  \end{equation}

  such that
  
  \begin{align}
    g_{s,t} &\leq G_{s,t} \\
    g_{s,t} &\geq 0 \\
    \sum_s g_{s,t} &= d_t
  \end{align}

In [121]:
# Define the objective function, include CO2 price of 120 euros/tCO2
co2_price = 120
m_co2.cost = Objective(expr=sum(marginal_price(s,co2_price)*m_co2.generators[s] for s in m_co2.S),
                      sense=pyo.minimize)

In [122]:
m_co2.cost.pprint()

cost : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 82.2*generators[CCGT] + 157.13*generators[GT_GasOil] + 1.5*generators[Hydro] + 117.93*generators[Coal] + 141.79*generators[Lignite] + 6.15*generators[Nuclear] + 1.4*generators[Wind] + generators[Solar]


Create dictionary of models for all timesteps

In [123]:
models_co2 = {}
for i in range(len(duration)):
    models_co2[duration['timestep'][i]] = m_co2.clone() #clone original model for each of the timesteps

#create a list of timesteps
timesteps = list(models_co2.keys())

Each timestep now has a model

In [124]:
models_co2

{'t1': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354262d10>,
 't2': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3542893b0>,
 't3': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354289450>,
 't4': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354292180>,
 't5': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3540c64a0>,
 't6': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354232400>,
 't7': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354292810>,
 't8': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354292bd0>,
 't9': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f354299040>,
 't10': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1f3542993b0>}

Add demand and generator limit constraints to all timesteps

  \begin{align}
    g_{s,t} &\leq G_{s,t} \\
    g_{s,t} &\geq 0 \\
    \sum_s g_{s,t} &= d_t
  \end{align}

In [125]:
for i in timesteps:
    model = models_co2[i]
    #add generator limit: capacity*cf at a given timestep
    
    @model.Constraint(model.S)
    def generator_limit(model, s):
        return model.generators[s] <= cf[cf['tech'] == s][i].values[0]*tech_data[tech_data['tech'] == s].cap_MW.values[0] 
    models_co2[i] = model
    
    #add demand constraint: sum of generation must equal demand    
    models_co2[i].demand_constraint = Constraint(expr=sum(models_co2[i].generators[s] for s in models_co2[i].S) == demand[demand.timestep == i].load_MW.values[0]) 

Solve the models at all timesteps

In [126]:
for i in timesteps:
    SolverFactory('cbc').solve(models_co2[i]).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5611431.71
  Upper bound: 5611431.71
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 9
  Number of nonzeros: 8
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number o

Extract optimum dispatch in MW for each timestep from the results

In [127]:
dispatch_co2 = pd.Series(models_co2[timesteps[0]].generators.get_values(),name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    d = pd.Series(models_co2[i].generators.get_values(),name=i).to_frame()
    dispatch_co2 = dispatch_co2.join(d)

dispatch_co2.to_csv('results/dispatch_co2.csv')
dispatch_co2 #MW

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,17700.0,18900.0,17700.0,18300.0,20100.0,18300.0,18900.0,20100.0,11793.0,18136.0
GT_GasOil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hydro,4056.0,4108.0,3588.0,3848.0,3692.0,4316.0,4160.0,3640.0,4056.0,4264.0
Coal,18450.0,20700.0,19800.0,11103.0,2560.0,616.0,2450.0,1618.0,0.0,0.0
Lignite,13399.0,7312.0,1948.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nuclear,7980.0,8064.0,7308.0,7476.0,7728.0,7896.0,8148.0,7308.0,7392.0,7728.0
Wind,10370.0,9760.0,12200.0,12810.0,18910.0,18910.0,12810.0,14640.0,20740.0,10370.0
Solar,11160.0,2325.0,4185.0,7905.0,7440.0,6975.0,5580.0,1395.0,0.0,0.0


Calculate dispatch expenditure [€] for each timestep: electricity price [€/MWh] * load [MW] * length of timestep [h]

In [128]:
dispatch_costs_co2 = duration.copy()
dispatch_costs_co2['load_MW'] = demand['load_MW']
dispatch_costs_co2['electricity_price'] = [models_co2[t].dual[models_co2[t].demand_constraint] for t in timesteps]
dispatch_costs_co2['expenditure'] = [models_co2[t].cost.expr() for t in timesteps]*dispatch_costs_co2['length']
dispatch_costs_co2.to_csv('results/dispatch_costs_co2.csv')
dispatch_costs_co2

,timestep,length,load_MW,electricity_price,expenditure
0,t1,102,83115,141.79,5.723660e+08
1,t2,962,71169,141.79,4.909321e+09
2,t3,962,66729,141.79,3.980518e+09
3,t4,962,61442,117.93,2.781359e+09
4,t5,962,60430,117.93,1.963538e+09
5,t6,962,57013,117.93,1.602104e+09
6,t7,962,52048,117.93,1.849323e+09
7,t8,962,48701,117.93,1.842544e+09
8,t9,962,43981,82.20,1.010067e+09
9,t10,962,40498,82.20,1.499970e+09


Calculate total annual expenditure and average electricity price

In [129]:
print("Total annual expenditure with 120€/tCO2 price [billion €]:", round((dispatch_costs_co2.expenditure).sum()/1e9,3))
print("Average electricity price with 120€/tCO2 price [€/MWh]:", round(((dispatch_costs_co2.electricity_price*dispatch_costs.length).sum()/dispatch_costs.length.sum()),2))

Total annual expenditure with 120€/tCO2 price [billion €]: 22.011
Average electricity price with 120€/tCO2 price [€/MWh]: 115.6


Calculate dispatch emissions per timestep in tCO2: (EMF [tCO2/MWh_th]/eta [MW_el/MW_th] * dispatch [MW])) * length of timestep [h]

In [130]:
dispatch_emissions_co2 = dispatch_co2.copy()

In [131]:
for i in timesteps:
    dispatch_emissions_co2[i] = (tech_data.set_index(dispatch_co2.index)['emf'].values*dispatch_co2[i]/tech_data.set_index(dispatch_co2.index)['eta'])*duration[duration['timestep'] == 't1'].length.values[0]

dispatch_emissions_co2.to_csv('results/dispatch_emissions_co2.csv')
dispatch_emissions_co2  #total emissions per timestep, tCO2

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,6.847147e+05,7.311360e+05,6.847147e+05,707925.333333,777557.333333,707925.333333,731136.0,777557.333333,456205.653333,701581.084444
GT_GasOil,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Hydro,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Coal,1.532404e+06,1.719283e+06,1.644531e+06,922183.457143,212626.285714,51163.200000,203490.0,134386.457143,0.000000,0.000000
Lignite,1.476034e+06,8.054899e+05,2.145917e+05,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Nuclear,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Wind,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
Solar,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [132]:
print('Total emissions with 120€/tCO2 price [million tCO2]:', round(dispatch_emissions_co2.sum().sum()/1e6,2))

Total emissions with 120€/tCO2 price [million tCO2]: 15.88


Calculate dual prices/shadow prices for each generator for all the timesteps: (how much more expensive the objective function would be if you increased the capacity by one unit)

In [133]:
dual_prices_co2 = pd.Series({s: models_co2[timesteps[0]].dual[models_co2[timesteps[0]].generator_limit[s]] for s in models_co2[timesteps[0]].S},name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    dp = pd.Series({s: models_co2[i].dual[models_co2[i].generator_limit[s]] for s in models_co2[i].S},name=i).to_frame()
    dual_prices_co2 = dual_prices_co2.join(dp)

dual_prices_co2.to_csv('results/dual_prices_co2.csv')
dual_prices_co2

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,-59.59,-59.59,-59.59,-35.73,-35.73,-35.73,-35.73,-35.73,0.00,0.00
GT_GasOil,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Hydro,-140.29,-140.29,-140.29,-116.43,-116.43,-116.43,-116.43,-116.43,-80.70,-80.70
Coal,-23.86,-23.86,-23.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Lignite,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Nuclear,-135.64,-135.64,-135.64,-111.78,-111.78,-111.78,-111.78,-111.78,-76.05,-76.05
Wind,-140.39,-140.39,-140.39,-116.53,-116.53,-116.53,-116.53,-116.53,-80.80,-80.80
Solar,-140.79,-140.79,-140.79,-116.93,-116.93,-116.93,-116.93,-116.93,-81.20,-81.20
